In [1]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as data
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import f1_score
from torch.utils.data import DataLoader
from tqdm import tqdm  # Import tqdm for progress bar
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

class EmotionDataset(data.Dataset):
    def __init__(self, root_dir, challenge, sequence_length=10, window_size=10):
        self.root_dir = root_dir
        self.challenge = challenge
        self.sequence_length = sequence_length
        self.window_size = window_size
        self.data = []
        self.labels = []
        
        # Load data
        for file in tqdm(os.listdir(root_dir), desc=f"Loading {challenge} dataset"):
            if file.endswith(".txt"):
                file_path = os.path.join(root_dir, file)
                try:
                    features = np.loadtxt(file_path, delimiter=",", dtype=np.float32)
                except ValueError as e:
                    print(f"Error loading {file}: {e}")
                    continue
                
                if features.size == 0:
                    continue
                
                if len(features.shape) == 1:
                    features = features.reshape(1, -1)
                
                if self.challenge == 'VA':
                    labels = features[:, -2:]
                elif self.challenge == 'EXPR':
                    labels = features[:, -1].astype(int)
                elif self.challenge == 'AU':
                    labels = features[:, -12:]
                else:
                    raise ValueError("Invalid challenge type")
                
                # Filtering conditions
                filtered_features = []
                filtered_labels = []
                for i, label in enumerate(labels):
                    if self.challenge == 'VA' and (-5 in label):
                        continue
                    if self.challenge == 'EXPR' and label == -1:
                        continue
                    if self.challenge == 'AU' and (-1 in label):
                        continue
                    if self.challenge == 'EXPR':
                        filtered_features.append(features[i, :-1])
                        filtered_labels.append(label)
                    else:
                        filtered_features.append(features[i, :-len(label)])
                        filtered_labels.append(label)
                if filtered_features:
                    self.data.append(np.array(filtered_features))
                    self.labels.append(np.array(filtered_labels))
        
        

        # For EXPR challenge, convert labels to one-hot encoding
        if self.challenge == 'EXPR':
            self.labels = [np.eye(8)[label] for label in self.labels]

        # Convert the lists to numpy arrays after filtering
        self.data = np.vstack(self.data) if self.data else np.array([])
        self.labels = np.vstack(self.labels) if self.labels else np.array([])
        
        # Create sequences with temporal coherence using window size and sequence length
        self.sequences = []
        self.sequence_labels = []
        
        for i in range(0, len(self.data) - self.sequence_length + 1, self.window_size):
            sequence_data = self.data[i:i+self.sequence_length]
            sequence_label = self.labels[i:i+self.sequence_length]  # Capture labels for all frames in sequence
            self.sequences.append(sequence_data)
            self.sequence_labels.append(sequence_label)
        
        self.sequences = np.array(self.sequences)
        self.sequence_labels = np.array(self.sequence_labels)
        
    def __len__(self):
        return len(self.sequences)

    def __getitem__(self, idx):
        return (torch.tensor(self.sequences[idx], dtype=torch.float32),
                torch.tensor(self.sequence_labels[idx], dtype=torch.float32))

In [ ]:
# Optimized CCC calculation
def CCC(y_true, y_pred):
    y_true_mean = torch.mean(y_true)
    y_pred_mean = torch.mean(y_pred)
    covariance = torch.mean((y_true - y_true_mean) * (y_pred - y_pred_mean))
    y_true_var = torch.var(y_true)
    y_pred_var = torch.var(y_pred)
    ccc = (2 * covariance) / (y_true_var + y_pred_var + (y_true_mean - y_pred_mean)**2 + 1e-8)
    return ccc

def CCC_loss(y_true, y_pred):
    if y_pred.shape[0] == 1:
        y_pred = y_pred.permute(1, 0, 2).reshape(-1, 2)
    if y_true.shape != y_pred.shape:
        raise ValueError(f"Shape mismatch: y_true {y_true.shape} vs y_pred {y_pred.shape}")
    valence_ccc = CCC(y_true[:, 0], y_pred[:, 0])
    arousal_ccc = CCC(y_true[:, 1], y_pred[:, 1])
    return 1 - 0.5 * (valence_ccc + arousal_ccc)

    

class LSTMModel(nn.Module):
    def __init__(self, input_dim=512, hidden_dim=128, sequence_length=10, num_layers=1,
                 train_val_arousal=False, train_emotions=False, train_actions=False):
        super(LSTMModel, self).__init__()
        
        self.sequence_length = sequence_length
        self.hidden_dim = hidden_dim
        self.num_layers = num_layers
        self.train_val_arousal = train_val_arousal
        self.train_emotions = train_emotions
        self.train_actions = train_actions
        
        # LSTM layer
        self.lstm = nn.LSTM(input_size=input_dim, hidden_size=hidden_dim, num_layers=num_layers, batch_first=True)
        
        # Fully connected layers
        self.dense = nn.Linear(hidden_dim, 512)
        self.relu = nn.ReLU()
        
        if self.train_val_arousal:
            self.val_arousal = nn.Linear(512, 2)
        if self.train_emotions:
            self.emotions = nn.Linear(512, 8)
        if self.train_actions:
            self.actions = nn.Linear(512, 12)
    
    def forward(self, x):
        lstm_out, (hn, cn) = self.lstm(x)  # (batch_size, sequence_length, hidden_dim)
        final_hidden_state = lstm_out
        x = torch.relu(self.dense(final_hidden_state))
        
        if self.train_val_arousal:
            return torch.tanh(self.val_arousal(x))
        if self.train_emotions:
            return torch.softmax(self.emotions(x), dim=2)
        if self.train_actions:
            return torch.sigmoid(self.actions(x))
        
        return x
        

import torch
import numpy as np
from tqdm import tqdm
from sklearn.metrics import f1_score

def find_best_thresholds(val_outputs, val_labels):
    """Find the best threshold for each element based on F1-score."""
    num_classes = val_outputs.shape[1]
    best_thresh = np.zeros(num_classes)
    search_thresholds = np.linspace(0.1, 0.9, num=9)  # Range of thresholds to test

    for i in range(num_classes):
        best_f1 = 0
        for thresh in search_thresholds:
            preds = (val_outputs[:, i] >= thresh).astype(int)
            f1 = f1_score(val_labels[:, i], preds)
            if f1 > best_f1:
                best_f1 = f1
                best_thresh[i] = thresh  # Update best threshold for this element

    return best_thresh

def train_model(model, train_loader, val_loader, criterion, optimizer, challenge):
    model.to(device)
    best_metric = -float('inf')
    best_thresholds = None  # Store best thresholds

    for epoch in range(10):
        model.train()
        train_loader_tqdm = tqdm(train_loader, desc=f"Training {challenge} Epoch {epoch+1}", ncols=100)
        for features, labels in train_loader_tqdm:
            features, labels = features.to(device, non_blocking=True), labels.to(device, non_blocking=True)
            optimizer.zero_grad()
            

            # Forward pass
            outputs = model(features)
            
            # Assuming outputs has shape (batch_size, channels, height, width)
            
            
            outputs = outputs.view(-1)
            labels = labels.view(-1)


            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

        # Validation loop
        model.eval()
        val_true, val_pred = [], []
        val_loader_tqdm = tqdm(val_loader, desc=f"Validating {challenge} Epoch {epoch+1}", ncols=100)
        with torch.no_grad():
            for features, labels in val_loader_tqdm:
                features, labels = features.to(device, non_blocking=True), labels.to(device, non_blocking=True)
                outputs = model(features)
                

                if challenge == 'VA':
                    outputs = outputs.view(-1, 2)
                    labels = labels.view(-1, 2)
                    val_true.append(labels.cpu().numpy())
                    val_pred.append(outputs.cpu().numpy())
                elif challenge == 'EXPR':
                    
                    
                    preds = torch.argmax(outputs, dim=2)
                    labels = torch.argmax(labels, dim=2)
                    
                    preds = preds.view(-1)
                    labels = labels.view(-1)
                    
                    
              
                    val_true.append(labels.cpu().numpy())
                    val_pred.append(preds.cpu().numpy())
                else:  # AU Detection with Optimized Thresholds
                    outputs = outputs.view(-1, 12)
                    labels = labels.view(-1, 12)
                    if best_thresholds is None:
                        preds = torch.round(outputs)  # Default to rounding initially
                    else:
                        thresholds_tensor = torch.tensor(best_thresholds, device=outputs.device)
                        preds = (outputs >= thresholds_tensor).float()
                    
                    val_true.append(labels.cpu().numpy())
                    val_pred.append(preds.cpu().numpy())
            
            # Compute validation metric
            val_true = np.concatenate(val_true, axis=0)
            val_pred = np.concatenate(val_pred, axis=0)
            
            if challenge == 'VA':
                valence_ccc = CCC(torch.tensor(val_true[:, 0]), torch.tensor(val_pred[:, 0]))
                arousal_ccc = CCC(torch.tensor(val_true[:, 1]), torch.tensor(val_pred[:, 1]))
                metric = 0.5 * (valence_ccc + arousal_ccc)
            elif challenge == 'EXPR':
                metric = f1_score(val_true, val_pred, average='macro')
            else:  # AU Detection
                metric = f1_score(val_true, val_pred, average='macro')
                print(best_thresholds)
                # Optimize thresholds after the first validation pass
                best_thresholds = find_best_thresholds(val_pred, val_true)

        print(f'Epoch {epoch+1}: Validation Metric ({challenge}): {metric:.4f}')

        print(f'Epoch {epoch+1}: Validation Metric ({challenge}): {metric:.4f}')

        if metric > best_metric:
            best_metric = metric
            model_filename = os.path.join("models_8th_temporal_DDAMFN", f"{challenge}_{metric:.4f}.pt")
            torch.save(model.state_dict(), model_filename)
            print(f"Model saved as {model_filename}")


    
import torch
import torch.nn as nn
import torch.nn.functional as F

# Check if the folder exists, create it if it doesn't
folder_path = "models_8th_temporal_DDAMFN"
if not os.path.exists(folder_path):
    os.makedirs(folder_path)

challenge_config = {
    'VA': {
        'output_dim': 2,
        'sequence_length': 10,
        'criterion': CCC_loss,
        'optimizer_lr': 0.00001,
        'train_val_arousal': True,
        'train_emotions': False,
        'train_actions': False,
        "window_size":10
    },
    'EXPR': {
        'output_dim': 8,
        'sequence_length': 10,
        'criterion': nn.CrossEntropyLoss(),
        'optimizer_lr': 0.00001,
        'train_val_arousal': False,
        'train_emotions': True,
        'train_actions': False,
        "window_size":5
    },
    'AU': {
        'output_dim': 12,
        'sequence_length': 10,
        'criterion': nn.BCELoss(),
        'optimizer_lr': 0.00001,
        'train_val_arousal': False,
        'train_emotions': False,
        'train_actions': True,
        "window_size":10
    }
}

# Iterate through the challenges
for challenge in ['EXPR', 'AU',"EXPR"]:
    config = challenge_config.get(challenge)

    # Load dataset and create data loaders
    train_set = EmotionDataset(f'Features/{challenge}/training_set_features', challenge, sequence_length=config['sequence_length'],window_size=config['window_size'])
    val_set = EmotionDataset(f'Features/{challenge}/validation_set_features', challenge, sequence_length=config['sequence_length'],window_size=config['window_size'])
    
    train_loader = DataLoader(train_set, batch_size=4, shuffle=True, num_workers=4, pin_memory=True)
    val_loader = DataLoader(val_set, batch_size=4, shuffle=False, num_workers=4, pin_memory=True)
    
    # Set input and output dimensions based on challenge
    input_dim = 512  # Each frame has 512 features
    hidden_dim = 128
    num_layers = 1
    
    # Create the LSTM model
    model = LSTMModel(
        input_dim=input_dim, 
        hidden_dim=hidden_dim, 
        sequence_length=config['sequence_length'], 
        num_layers=num_layers, 
        train_val_arousal=config['train_val_arousal'], 
        train_emotions=config['train_emotions'], 
        train_actions=config['train_actions']
    )

    # Set the loss function and optimizer
    criterion = config['criterion']
    optimizer = optim.Adam(model.parameters(), lr=1e-5)
    
    # Train the model
    train_model(model, train_loader, val_loader, criterion, optimizer, challenge)